In [1]:
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
import torch.nn.functional as F
import torch.nn as nn
import seaborn as sns
from scipy.optimize import minimize
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import matplotlib as mpl
from tqdm import tqdm
import multiprocessing
import threading

In [2]:
# 设置GPU设备，如果没有可用的GPU，则使用CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(888888)

print(device)

# 打印GPU型号
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f'Number of available GPUs: {num_gpus}')
    for i in range(num_gpus):
        print(f'GPU {i}: {torch.cuda.get_device_name(i)}')
else:
    print('No GPU available, using CPU')

cpu
No GPU available, using CPU


In [3]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(2, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

In [4]:
# 基础参数
epochs = 201    # 训练代数
N = 100    # 内点配置点数
N1 = 0.1    # 边界点配置点数
N2 = 1000    # PDE数据点

# 需要改的参数

20 50 100 200 500 1000

In [5]:
# 论文参数
h = 200      # 网格分辨率1/h=0.05
delta = 0.002   # 小区域的δ值

# 迭代步骤

In [6]:
# 初始化两个模型
model1 = MLP().to(device)
model2 = MLP().to(device)

# 加载权重
model1.load_state_dict(torch.load('weights1_+.pth', map_location=device))
model2.load_state_dict(torch.load('weights2_+.pth', map_location=device))

# 输入数据--右
x = delta
y = torch.linspace(0, 1, steps=h).to(device)  # 这里选择h个点，可以根据需要调整
input_data_right = torch.stack([torch.full_like(y, x), y], dim=1)

# 输入数据--左
x = -delta
y = torch.linspace(0, 1, steps=h).to(device)  # 选择h个点
input_data_left = torch.stack([torch.full_like(y, x), y], dim=1)

input_data_left = input_data_left.to(device)
input_data_right = input_data_right.to(device)

FileNotFoundError: [Errno 2] No such file or directory: 'weights1_+.pth'

In [203]:
# 添加n次的平均迭代次数与时间，改Adam
ave_count = ave_time = 0
min_count = min_time = 99999
current_time = time.time()
c = 0
for i in range(1):
    c += 1
    count = 0      # 迭代计数器
    # 重新导入
    model1.load_state_dict(torch.load('weights1.pth', map_location=device))
    model2.load_state_dict(torch.load('weights2.pth', map_location=device))
    while 1:
        # 记录迭代次数
        count += 1

        # 计算左右两直线
        model1.eval()
        model2.eval()
        with torch.no_grad():
            output1 = model1(input_data_left)             # 左直线
            output2 = model2(input_data_right)            # 右直线


        # 进行对区域1的训练
        optimizer = optim.Adam(model1.parameters(), lr=0.00001)
        for epoch in range(epochs):
            model1.train()
            optimizer.zero_grad()
            output = model1(input_data_right)               
            loss = torch.sum(torch.abs(output - output2))  # 重新计算损失
            loss.backward()
            optimizer.step()
            
            
        # 进行对区域2的训练
        optimizer = optim.Adam(model2.parameters(), lr=0.00001)
        for epoch in range(epochs):
            model2.train()
            optimizer.zero_grad()
            output = model2(input_data_left)                           # 重新计算output2
            loss = torch.sum(torch.abs(output1 - output))              # 重新计算损失
            loss.backward()
            optimizer.step()


        # 退出操作
        model1.eval()
        model2.eval()
        with torch.no_grad():
            result_left = model1(input_data_left)
            result_right = model2(input_data_right)
        result = torch.abs(torch.sum(result_left - result_right))
        # 收敛退出循环
        if (result / count) < 1e-3:
            T = abs(current_time-time.time())
            print(f'迭代结束!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!次数为{count},时间为：{T}s当前结果为：{result / count}')
            current_time = time.time()
            break
        # 隔10次打印结果
        elif count % 1 == 0:
            print(f'第{count}次迭代，结果：{result / count}')
            # print(result_left.T - result_right.T)
            print('-----------------------------------------------------------------------------')

    if min_count > count and count != 0:
        min_count = count
    if min_time > T and count != 0:
        min_time = T
        
    ave_time += T
    ave_count += count
    print('***************************************************************************')

# 最终结果
print(f'迭代次数：{min_count},使用时间：{min_time}s')

第1次迭代，结果：102.66362762451172
-----------------------------------------------------------------------------
第2次迭代，结果：54.145301818847656
-----------------------------------------------------------------------------
第3次迭代，结果：30.5867919921875
-----------------------------------------------------------------------------
第4次迭代，结果：19.653488159179688
-----------------------------------------------------------------------------
第5次迭代，结果：14.147781372070312
-----------------------------------------------------------------------------
第6次迭代，结果：10.817544937133789
-----------------------------------------------------------------------------
第7次迭代，结果：8.481889724731445
-----------------------------------------------------------------------------
第8次迭代，结果：6.7563018798828125
-----------------------------------------------------------------------------
第9次迭代，结果：5.400205135345459
-----------------------------------------------------------------------------
第10次迭代，结果：4.310580730438232
----------------------

KeyboardInterrupt: 